In [1]:
!pip install PySastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 1.7 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import os
import pickle
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB

#### Import Dataset

In [2]:
data_path = '/content/dataset'
# data_path = '...\Flask-server\dataset'

labels = []
tokens = []
titles = []

for f in os.listdir(data_path):
    full_path = os.path.join(data_path, f)
    if os.path.isfile(full_path):
        titles.append(f.split('.')[0])
        with open(full_path, 'r', encoding='utf8', errors='ignore') as infile:
            for line in infile:
                tokens.append(line.strip())
                labels.append(len(titles) - 1)

df = pd.DataFrame(list(zip(tokens, labels)), columns=['sent', 'label'])

In [3]:
df

,sent,label
0,adalah bagian dari aplikasi yang berfungsi unt...,0
1,Pengembangan mencakup pembuatan API yang memun...,0
2,mengelola penyimpanan data dan operasi yang di...,0
3,Pengembang bekerja dengan bahasa pemrograman s...,0
4,Arsitektur sering melibatkan penggunaan server...,0
...,...,...
1783,Firebase adalah platform dari Google yang meny...,6
1784,Firebase memungkinkan integrasi fitur seperti ...,6
1785,Firebase menyediakan Firestore dan Realtime Da...,6
1786,Firebase mendukung autentikasi pengguna dengan...,6


In [4]:
titles

['backend', 'ai', 'devops', 'frontend', 'gamedev', 'ios', 'android']

In [5]:
df.sample(20)

,sent,label
930,Kemampuan dalam menulis kode yang dapat diakse...,3
1496,TestFlight,5
1486,SwiftUI memungkinkan pembuatan antarmuka aplik...,5
1310,Anda akan sering berkolaborasi dengan desainer...,5
166,Pengembang backend harus bisa menangani dan me...,0
1578,Pemrograman melibatkan penanganan berbagai jen...,6
791,UI yang baik sangat bergantung pada pemahaman ...,3
643,Tim sering menggunakan metode-metode terbaru u...,2
1339,Pemrograman iOS memungkinkan Anda membuat apli...,5
1125,Pengembang harus memahami bagaimana mengelola...,4


#### Data Cleaning

In [6]:
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stop_factory = StopWordRemoverFactory()
stop_words = stop_factory.get_stop_words()

In [7]:
def preprocess_text(text):
    cleaned_text = " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", text).split() if word.lower() not in stop_words])
    return cleaned_text.lower()

df['cleaned'] = df['sent'].apply(preprocess_text)


In [8]:
df

,sent,label,cleaned
0,adalah bagian dari aplikasi yang berfungsi unt...,0,aplikasi fungsi kelola data logika bisnis server
1,Pengembangan mencakup pembuatan API yang memun...,0,kembang cakup buat api komunikasi frontend dat...
2,mengelola penyimpanan data dan operasi yang di...,0,kelola simpan data operasi server jadi pusat p...
3,Pengembang bekerja dengan bahasa pemrograman s...,0,kembang bahasa pemrograman server side python ...
4,Arsitektur sering melibatkan penggunaan server...,0,arsitektur libat guna server database sistem m...
...,...,...,...
1783,Firebase adalah platform dari Google yang meny...,6,firebase platform google sedia layan backend a...
1784,Firebase memungkinkan integrasi fitur seperti ...,6,firebase integrasi fitur notifikasi push anali...
1785,Firebase menyediakan Firestore dan Realtime Da...,6,firebase sedia firestore realtime database sim...
1786,Firebase mendukung autentikasi pengguna dengan...,6,firebase dukung autentikasi guna metode email ...


In [9]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

tfidf = TfidfVectorizer(max_features=1400)

X_tfidf = tfidf.fit_transform(df['cleaned'])

selector = SelectKBest(chi2, k=1000)
X_selected = selector.fit_transform(X_tfidf, df['label'])

In [10]:
print(X_tfidf.shape)

(1788, 1400)


In [11]:
print(X_selected.shape)

(1788, 1000)


#### Data Training

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X_selected, df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [13]:
# model = LogisticRegression()
# model = MultinomialNB()
model = BernoulliNB()


pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1400)),
    ('select', SelectKBest(chi2, k=1000)),
    ('classifier', BernoulliNB())
])

In [14]:
pipeline.fit(df['cleaned'], df['label'])
model.fit(x_train, y_train)

BernoulliNB()

In [15]:
y_pred = model.predict(x_test)

print(classification_report(y_test, y_pred))

print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.73      0.88      0.80        52
           1       1.00      0.91      0.95        53
           2       0.89      0.86      0.88        49
           3       0.91      0.78      0.84        51
           4       0.83      0.98      0.90        51
           5       0.92      0.90      0.91        49
           6       0.90      0.81      0.85        53

    accuracy                           0.87       358
   macro avg       0.88      0.87      0.88       358
weighted avg       0.88      0.87      0.88       358

[[46  0  4  0  0  1  1]
 [ 0 48  0  1  3  0  1]
 [ 6  0 42  1  0  0  0]
 [ 6  0  0 40  2  1  2]
 [ 0  0  0  1 50  0  0]
 [ 2  0  0  1  1 44  1]
 [ 3  0  1  0  4  2 43]]


In [16]:
def predict_text(text):
    cleaned_text = preprocess_text(text)
    prediction = pipeline.predict([cleaned_text])
    return titles[prediction[0]]

In [17]:
new_text = "saya mempunyai pengalaman melakukan pemrograman pada produk apple"
predicted_label = predict_text(new_text)
print(predicted_label)

ios


In [18]:
new_text = "bapak jokowi adalah idola saya. saya ingin bisa menjadi pemrogram yang bekerja di bidang server seperti bapak jokowi"
predicted_label = predict_text(new_text)
print(predicted_label)

backend


In [19]:
new_text = "saya mempunyai pengalaman melakukan pemrograman pada produk apple"
predicted_label = predict_text(new_text)
print(predicted_label)

ios


In [20]:
new_text = "bapak jokowi adalah idola saya. saya ingin bisa menjadi pemrogram yang bekerja di bidang server seperti bapak jokowi"
predicted_label = predict_text(new_text)
print(predicted_label)

backend


In [21]:
new_text = "saya tertarik untuk belajar di bidang pengembangan aplikasi google"
predicted_label = predict_text(new_text)
print(predicted_label)

android


In [22]:
new_text = "bapak jokowi adalah idola saya. saya ingin bisa menjadi pemrogram yang bekerja di bidang server seperti bapak jokowi"
predicted_label = predict_text(new_text)
print(predicted_label)

backend


In [23]:
new_text = "saya mempunyai pengalaman mengerjakan project dengan bahasa python dan pernah membuat project prediksi harga"
predicted_label = predict_text(new_text)
print(predicted_label)

ai


In [24]:
new_text = "saya mempunyai pengalaman mengerjakan project css"
predicted_label = predict_text(new_text)
print(predicted_label)

frontend


In [25]:
df['label'].values

array([0, 0, 0, ..., 6, 6, 6])

In [26]:
cl = "bernoulli"
v = 8
titles = ['backend', 'ai', 'devops', 'frontend', 'gamedev', 'ios', 'android']
labels = df['label'].values

# Menyimpan model dengan format nama yang diberikan
filename = f'{cl}_model_v{v}_c{len(titles)}_e{int(len(labels)/len(titles))}.pickle'
with open(filename, 'wb') as f:
    pickle.dump(model, f)

print(f"Model saved as {filename}")

Model saved as bernoulli_model_v8_c7_e255.pickle


In [28]:
filename = 'tfidf_vectorizer_v8.pkl'
with open(filename, 'wb') as f:
    pickle.dump(tfidf, f)

print(f"Model saved as {filename}")

Model saved as tfidf_vectorizer_v8.pkl


In [29]:
filename = 'select_kbest_v8.pkl'
with open(filename, 'wb') as f:
    pickle.dump(selector, f)

print(f"Model saved as {filename}")

Model saved as select_kbest_v8.pkl
